In [88]:
from riotwatcher import LolWatcher, ApiError
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from datetime import datetime
import json

# current date
now = datetime.now()

date = now.strftime("%d %b, %Y")

lw = LolWatcher('RGAPI-0bb964a5-de77-4126-93cb-dbb50ea64a0f')

summ_list = [
    'GrandfatherFlesh','satorian','Superpen1sball',
    '36CYOB','Rumahoy','Tobias Fate','SoloRenektonOnly',
    'shaanathan','be pure','Mistress Lolly']

my_region = 'americas'
filePath = "/Users/danielmoran/Documents/School/ML/MLProject_2021/Dataset/" + date
print(filePath)
Games=[]

for summ in summ_list:
    me = lw.summoner.by_name('na1', summ)
    last_game = 0
    
    #START INDEX
    #12/4 - 0,100,200
    #12/5 - 300,400,500
    #12/6 - 600,700,800
    
    my_matches = lw.match.matchlist_by_puuid('americas', me['puuid'], type="normal", count = 100)
    my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], type="ranked", count = 100)
    my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], type="normal", start = 100, count = 100)
    my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], type="normal", start = 100, count = 100)
    my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], type="normal", start = 200, count = 100)
    my_matches += lw.match.matchlist_by_puuid('americas', me['puuid'], type="normal", start = 200, count = 100)
    
    n_games = 300
    j=0
    count=0

    while count < n_games:
        try:
            time.sleep(1)
            last_match = my_matches[count]
            print(summ, " : ", count)
            count+=1
            #print(last_match)
            game_info = lw.match.by_id(my_region, last_match)
            gameMode = game_info['info']['gameMode']
            if gameMode == "CLASSIC":
                winner = game_info['info']['teams'][0]['win']
                #print(winner)
                #print("goldP1 : \n", lw.match.timeline_by_match(my_region, last_match)['info']['frames'][-1]['participantFrames']['1']['totalGold'])
                data = lw.match.timeline_by_match(my_region, last_match)['info']['frames']

                for event in data:
                    #print(type(event),'\n',type(event.items()),event.items())
                    if "participantFrames" in event:
                        #print('\n\n\n',"EVENT : ",type(event),'\n',event)
                        #print('\n\n\n',"TS : ",event['timestamp'])
                        if event['timestamp'] <= 900000:
                            selection = event
                        if event['timestamp'] > 900000:
                            if abs(selection['timestamp']-900000) > abs(event['timestamp']-900000):
                                selection = event
                                break
                            else:
                                break
                event_data = [selection['participantFrames'], selection['timestamp']]
                #print(json.dumps(event_data, indent=2))

                frame = event_data[0]
                game = np.zeros(31)
                i = 0

                for player in range(1, 11):
                    game[i] = frame[str(player)]['damageStats']['totalDamageDoneToChampions']
                    i+=1
                    game[i] = frame[str(player)]['totalGold']
                    i+=1
                    game[i] = frame[str(player)]['xp']
                    i+=1
                if winner == True and player <= 5:
                    game[i] = 1
                else:
                    game[i] = 0

                #print(game)
                Games.append(game)
        except:
            count+=1

dataset = pd.DataFrame(Games, columns = [
                        'Player_1_Damage','Player_1_Gold','Player_1_XP',
                        'Player_2_Damage','Player_2_Gold', 'Player_2_XP',
                        'Player_3_Damage','Player_3_Gold', 'Player_3_XP',
                        'Player_4_Damage','Player_4_Gold', 'Player_4_XP',
                        'Player_5_Damage','Player_5_Gold', 'Player_5_XP',
                        'Player_6_Damage','Player_6_Gold', 'Player_6_XP',
                        'Player_7_Damage','Player_7_Gold', 'Player_7_XP',
                        'Player_8_Damage','Player_8_Gold', 'Player_8_XP',
                        'Player_9_Damage','Player_9_Gold', 'Player_9_XP',
                        'Player_10_Damage','Player_10_Gold', 'Player_10_XP',
                        'Victory'])

dataset.head()

dataset.to_csv(filePath)

/Users/danielmoran/Documents/School/ML/MLProject_2021/Dataset/04 Dec, 2021
GrandfatherFlesh  :  0
GrandfatherFlesh  :  1
GrandfatherFlesh  :  2
GrandfatherFlesh  :  3
GrandfatherFlesh  :  4
GrandfatherFlesh  :  5
GrandfatherFlesh  :  6
GrandfatherFlesh  :  7
GrandfatherFlesh  :  8
GrandfatherFlesh  :  9
GrandfatherFlesh  :  10
GrandfatherFlesh  :  11
GrandfatherFlesh  :  12
GrandfatherFlesh  :  13
GrandfatherFlesh  :  14
GrandfatherFlesh  :  15
GrandfatherFlesh  :  16
GrandfatherFlesh  :  17
GrandfatherFlesh  :  18
GrandfatherFlesh  :  19
GrandfatherFlesh  :  20
GrandfatherFlesh  :  21
GrandfatherFlesh  :  22
GrandfatherFlesh  :  23
GrandfatherFlesh  :  24
GrandfatherFlesh  :  25
GrandfatherFlesh  :  26
GrandfatherFlesh  :  27
GrandfatherFlesh  :  28
GrandfatherFlesh  :  29
GrandfatherFlesh  :  30
GrandfatherFlesh  :  31
GrandfatherFlesh  :  32
GrandfatherFlesh  :  33
GrandfatherFlesh  :  34
GrandfatherFlesh  :  35
GrandfatherFlesh  :  36
GrandfatherFlesh  :  37
GrandfatherFlesh  :  38

satorian  :  47
satorian  :  48
satorian  :  49
satorian  :  50
satorian  :  51
satorian  :  52
satorian  :  53
satorian  :  54
satorian  :  55
satorian  :  56
satorian  :  57
satorian  :  58
satorian  :  59
satorian  :  60
satorian  :  61
satorian  :  62
satorian  :  63
satorian  :  64
satorian  :  65
satorian  :  66
satorian  :  67
satorian  :  68
satorian  :  69
satorian  :  70
satorian  :  71
satorian  :  72
satorian  :  73
satorian  :  74
satorian  :  75
satorian  :  76
satorian  :  77
satorian  :  78
satorian  :  79
satorian  :  80
satorian  :  81
satorian  :  82
satorian  :  83
satorian  :  84
satorian  :  85
satorian  :  86
satorian  :  87
satorian  :  88
satorian  :  89
satorian  :  90
satorian  :  91
satorian  :  92
satorian  :  93
satorian  :  94
satorian  :  95
satorian  :  96
satorian  :  97
satorian  :  98
satorian  :  99
satorian  :  100
satorian  :  101
satorian  :  102
satorian  :  103
satorian  :  104
satorian  :  105
satorian  :  106
satorian  :  107
satorian  :  108

Superpen1sball  :  177
Superpen1sball  :  178
Superpen1sball  :  179
Superpen1sball  :  180
Superpen1sball  :  181
Superpen1sball  :  182
Superpen1sball  :  183
Superpen1sball  :  184
Superpen1sball  :  185
Superpen1sball  :  186
Superpen1sball  :  187
Superpen1sball  :  188
Superpen1sball  :  189
Superpen1sball  :  190
Superpen1sball  :  191
Superpen1sball  :  192
Superpen1sball  :  193
Superpen1sball  :  194
Superpen1sball  :  195
Superpen1sball  :  196
Superpen1sball  :  197
Superpen1sball  :  198
Superpen1sball  :  199
Superpen1sball  :  200
Superpen1sball  :  201
Superpen1sball  :  202
Superpen1sball  :  203
Superpen1sball  :  204
Superpen1sball  :  205
Superpen1sball  :  206
Superpen1sball  :  207
Superpen1sball  :  208
Superpen1sball  :  209
Superpen1sball  :  210
Superpen1sball  :  211
Superpen1sball  :  212
Superpen1sball  :  213
Superpen1sball  :  214
Superpen1sball  :  215
Superpen1sball  :  216
Superpen1sball  :  217
Superpen1sball  :  218
Superpen1sball  :  219
Superpen1sb

Tobias Fate  :  4
Tobias Fate  :  5
Tobias Fate  :  6
Tobias Fate  :  7
Tobias Fate  :  8
Tobias Fate  :  9
Tobias Fate  :  10
Tobias Fate  :  11
Tobias Fate  :  12
Tobias Fate  :  13
Tobias Fate  :  14
Tobias Fate  :  15
Tobias Fate  :  16
Tobias Fate  :  17
Tobias Fate  :  18
Tobias Fate  :  19
Tobias Fate  :  20
Tobias Fate  :  21
Tobias Fate  :  22
Tobias Fate  :  23
Tobias Fate  :  24
Tobias Fate  :  25
Tobias Fate  :  26
Tobias Fate  :  27
Tobias Fate  :  28
Tobias Fate  :  29
Tobias Fate  :  30
Tobias Fate  :  31
Tobias Fate  :  32
Tobias Fate  :  33
Tobias Fate  :  34
Tobias Fate  :  35
Tobias Fate  :  36
Tobias Fate  :  37
Tobias Fate  :  38
Tobias Fate  :  39
Tobias Fate  :  40
Tobias Fate  :  41
Tobias Fate  :  42
Tobias Fate  :  43
Tobias Fate  :  44
Tobias Fate  :  45
Tobias Fate  :  46
Tobias Fate  :  47
Tobias Fate  :  48
Tobias Fate  :  49
Tobias Fate  :  50
Tobias Fate  :  51
Tobias Fate  :  52
Tobias Fate  :  53
Tobias Fate  :  54
Tobias Fate  :  55
Tobias Fate  :  56

SoloRenektonOnly  :  260
SoloRenektonOnly  :  261
SoloRenektonOnly  :  262
SoloRenektonOnly  :  263
SoloRenektonOnly  :  264
SoloRenektonOnly  :  265
SoloRenektonOnly  :  266
SoloRenektonOnly  :  267
SoloRenektonOnly  :  268
SoloRenektonOnly  :  269
SoloRenektonOnly  :  270
SoloRenektonOnly  :  271
SoloRenektonOnly  :  272
SoloRenektonOnly  :  273
SoloRenektonOnly  :  274
SoloRenektonOnly  :  275
SoloRenektonOnly  :  276
SoloRenektonOnly  :  277
SoloRenektonOnly  :  278
SoloRenektonOnly  :  279
SoloRenektonOnly  :  280
SoloRenektonOnly  :  281
SoloRenektonOnly  :  282
SoloRenektonOnly  :  283
SoloRenektonOnly  :  284
SoloRenektonOnly  :  285
SoloRenektonOnly  :  286
SoloRenektonOnly  :  287
SoloRenektonOnly  :  288
SoloRenektonOnly  :  289
SoloRenektonOnly  :  290
SoloRenektonOnly  :  291
SoloRenektonOnly  :  292
SoloRenektonOnly  :  293
SoloRenektonOnly  :  294
SoloRenektonOnly  :  295
SoloRenektonOnly  :  296
SoloRenektonOnly  :  297
SoloRenektonOnly  :  298
SoloRenektonOnly  :  299


be pure  :  109
be pure  :  110
be pure  :  111
be pure  :  112
be pure  :  113
be pure  :  114
be pure  :  115
be pure  :  116
be pure  :  117
be pure  :  118
be pure  :  119
be pure  :  120
be pure  :  121
be pure  :  122
be pure  :  123
be pure  :  124
be pure  :  125
be pure  :  126
be pure  :  127
be pure  :  128
be pure  :  129
be pure  :  130
be pure  :  131
be pure  :  132
be pure  :  133
be pure  :  134
be pure  :  135
be pure  :  136
be pure  :  137
be pure  :  138
be pure  :  139
be pure  :  140
be pure  :  141
be pure  :  142
be pure  :  143
be pure  :  144
be pure  :  145
be pure  :  146
be pure  :  147
be pure  :  148
be pure  :  149
be pure  :  150
be pure  :  151
be pure  :  152
be pure  :  153
be pure  :  154
be pure  :  155
be pure  :  156
be pure  :  157
be pure  :  158
be pure  :  159
be pure  :  160
be pure  :  161
be pure  :  162
be pure  :  163
be pure  :  164
be pure  :  165
be pure  :  166
be pure  :  167
be pure  :  168
be pure  :  169
be pure  :  170
be pure 

Mistress Lolly  :  229
Mistress Lolly  :  230
Mistress Lolly  :  231
Mistress Lolly  :  232
Mistress Lolly  :  233
Mistress Lolly  :  234
Mistress Lolly  :  235
Mistress Lolly  :  236
Mistress Lolly  :  237
Mistress Lolly  :  238
Mistress Lolly  :  239
Mistress Lolly  :  240
Mistress Lolly  :  241
Mistress Lolly  :  242
Mistress Lolly  :  243
Mistress Lolly  :  244
Mistress Lolly  :  245
Mistress Lolly  :  246
Mistress Lolly  :  247
Mistress Lolly  :  248
Mistress Lolly  :  249
Mistress Lolly  :  250
Mistress Lolly  :  251
Mistress Lolly  :  252
Mistress Lolly  :  253
Mistress Lolly  :  254
Mistress Lolly  :  255
Mistress Lolly  :  256
Mistress Lolly  :  257
Mistress Lolly  :  258
Mistress Lolly  :  259
Mistress Lolly  :  260
Mistress Lolly  :  261
Mistress Lolly  :  262
Mistress Lolly  :  263
Mistress Lolly  :  264
Mistress Lolly  :  265
Mistress Lolly  :  266
Mistress Lolly  :  267
Mistress Lolly  :  268
Mistress Lolly  :  269
Mistress Lolly  :  270
Mistress Lolly  :  271
Mistress Lo